![](https://www.nuplan.org/static/media/nuPlan_final.3fde7586.png)

### Contents

1. [Introduction to nuPlan](#introduction)
2. [Training an ML planner](#training)
3. [Simulating a planner](#simulation)
4. [Visualizing metrics and scenarios](#dashboard)

# Introduction to nuPlan <a name="introduction"></a>


Welcome to nuPlan! This notebook will explore the nuPlan simulation framework, training platform as well as the nuBoard metrics/scenarios visualization dashboard.

## What is nuPlan

nuPlan is the world’s first closed-loop ML-based planning benchmark for autonomous driving.

It provides a high quality dataset with 1500h of human driving data from 4 cities across the US and Asia with widely varying traffic patterns (Boston, Pittsburgh, Las Vegas and Singapore). In addition, it provides a closed-loop simulation framework with reactive agents, a training platform as well as a large set of both general and scenario-specific planning metrics.

![](https://www.nuscenes.org/static/media/framework_steps.2d4642df.png)

## Training & simulation framework

The nuPlan training and simulation framework aims to:
* create a simulation pipeline to evaluate a planner on large dataset with various scenarios
* score planner performance with common and scenario-dependent metrics
* compare planners based on measured metrics and provide intuitive visualizations
* train planners with the provided framework to allow quick implementation and iteration
* support closed-loop simulation and training

<br />

![](https://www.nuplan.org/static/media/planning_framework.ca3c2969.png)

## Scenarios in nuPlan

nuPlan aims to capture challenging yet representative scenarios from real-world encounters. This enables the benchmarking of planning systems both in expert imitation (open-loop) and reactive planning (closed-loop) settings.

These scenarios includes:
* highly interactive scenes with traffic participants (e.g. tailgating, high-velocity overtakes, double parked cars, jaywalking)
* various ego behaviors (e.g. vehicle following, yielding, lane merging) and dynamics (e.g. mixed speed profiles, abrupt braking, speed bumps, high jerk maneuvers)
* scene layouts of varied complexity (e.g. pudos, traffic/stop controlled intersections, unprotected turns) and temporary zones (e.g. construction areas)

The dataset is automatically tagged with scenario labels based on certain primitive attributes.
These scenario tags can then be used to extract representative metrics for the planner's evaluation.

Example mined scenarios in nuPlan:

| | | |
| :-: | :-: | :-: |
| Unprotected cross turn | Dense vehicle interactions | Jaywalker in front |
| ![](https://www.nuscenes.org/static/media/unprotected-cross.51feef7e.webp) | ![](https://www.nuscenes.org/static/media/dense-interactions.16de47ec.webp) | ![](https://www.nuscenes.org/static/media/jaywalker.03083823.webp) |
| Lane change | Ego at pickup/dropoff area | Ego following vehicle |
| ![](https://www.nuscenes.org/static/media/lane-change.54bfca1c.webp) | ![](https://www.nuscenes.org/static/media/pickup-dropoff.4dd1c418.webp) | ![](https://www.nuscenes.org/static/media/following-vehicle.4cacd559.webp) |

## Database

Download a database for training/simulation from [here](https://nuplan.org/nuplan#download).

| Database | Size | Duration | Num Logs | Cities | Num Scenarios | Sensor Data | Description |
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| nuplan_v0.1_mini (recommended) | ~5GB | 2.5h | 8 | Las Vegas | 14 | N/A | nuPlan teaser database (mini version) |
| nuplan_v0.1 | ~500GB | ~230h | 380 | Las Vegas | 14 | N/A | nuPlan teaser database |

The full 1500h dataset with driving data from 4 cities (Boston, Pittsburgh, Las Vegas and Singapore) as well as sensor data will be released in Q1 2022.

## Setup

To be able to access all resources within this notebook, make sure Jupyter is launched at the root of this repo. The path of the notebook should be `/notebook/<repo_root>`.

In [1]:
# (Optional) Increase notebook width for all embedded cells to display properly
from IPython.core.display import display, HTML
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_7532/4180358289.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Useful imports
import os
from pathlib import Path
import tempfile

import hydra

# Training an ML planner <a name="training"></a>

## Imitation learning
In the following section we will train an ML planning policy with the aim estimate the ego's future trajectory and control the vehicle.

The policy is learned through imitation learning, a supervised learning approach in which - in the context of autonomous driving - the behavior of an expert human driver is used as a target signal to supervise the model.

## Model features & targets
A planning policy consumes a set of episodic observations and encodes them through a deep neural network to regress a future trajectory.

The observations can be historic or present ego and agent poses as well as static/dynamic map information across different map layers.<br />
These signals can be encoded through various representations, such as raster or vector format for the map signal, each with their pros and cons for each model flavor.

Using these input features the model predicts a discretized future trajectory across a fixed time horizon.<br />
The trajectory consists of a set of discrete future states (position, heading and velocity) sampled at fixed intervals which express the likelihood of the vehicle being at that state in the future.<br />
For example, a predicted trajectory may consist of 10 future poses sampled at intervals of 0.5s across a 5s horizon.

## Learning objectives
The policy is trained to maximize a set of aggregated objectives such as imitation, collision avoidance, traffic rule violation etc.<br />
Imitation is the core training objective which indicates how close the predicted trajectory is to the expert ground truth and penalizes model predictions that deviate in space and time from the demonstration.

## Training parameters

The following parameter categories define the training protocol which includes the model, metrics, objectives etc.

A working example composition of these parameters can be found in the next section.

---

### ML models

Change the training model with `model=X` where `X` is a config yaml defined in the table below. 

| Model | Description | Config |
| --- | --- | --- |
| Raster model (CNN) | Raster-based model that uses a CNN backbone to encode ego, agent and map information as raster layers<br />Any (pretrained) backbone from the TIMM library can be used (e.g. ResNet50, EfficientNetB3) | `raster_model` |
| Vector model (LaneGCN) | Vector-based model that uses a series of MLPs to encode ego and agent signals, a lane graph to encode vector-map elements and a fusion network to capture lane & agent intra/inter-interactions through attention layers<br />Implementation of LaneGCN paper ("Learning Lane Graph Representations for Motion Forecasting") | `vector_model` |
| Simple vector model | Toy vector-based model that consumes ego, agent and lane signals through a series of MLPs | `simple_vector_model` |

<br />

### Training objectives

Change the training objectives with `objective=[X, ...]` where `X` is a config yaml defined in the table below. 

| Objective | Description | Config |
| --- | --- | --- |
| Imitation objective | Penalizes the predicted trajectory that deviates from the expert demonstration | `imitation_objective` |

<br />

### Training metrics

Change the training objectives with `training_metric=[X, ...]` where `X` is a config yaml defined in the table below. 

| Metric | Description | Config |
| --- | --- | --- |
| Average displacement error | RMSE translation error across full predicted trajectory | `avg_displacement_error` |
| Average heading error | RMSE heading error across full predicted trajectory | `avg_heading_error` |
| Final displacement error | L2 error of predicted trajectory's final pose translation | `final_displacement_error` |
| Final heading error | L2 error of predicted trajectory's final pose heading | `final_heading_error` |

## Prepare the training config

In [3]:
# Location of path with all training configs
CONFIG_PATH = '../nuplan/planning/script/config/training'
CONFIG_NAME = 'default_training'

# Create a temporary directory to store the cache and experiment artifacts
SAVE_DIR = Path('/home/fla/temp') / 'tutorial_nuplan_framework'  # optionally replace with persistent dir
EXPERIMENT = 'training_raster_experiment'
LOG_DIR = str(SAVE_DIR / EXPERIMENT)

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={str(SAVE_DIR)}',
    f'cache_dir={str(SAVE_DIR)}/cache',
    f'experiment_name={EXPERIMENT}',
    'py_func=train',
    '+training=training_raster_model',  # raster model that consumes ego, agents and map raster layers and regresses the ego's trajectory
    'scenario_builder=nuplan_mini',  # use nuplan mini database
    'scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=500',  # Choose 500 scenarios to train with
    'scenario_builder.nuplan.scenario_filter.subsample_ratio=0.01',  # subsample scenarios from 20Hz to 0.2Hz
    'lightning.trainer.params.accelerator=ddp_spawn',  # ddp is not allowed in interactive environment, using ddp_spawn instead - this can bottleneck the data pipeline, it is recommended to run training outside the notebook
    'lightning.trainer.params.max_epochs=10',
    'data_loader.params.batch_size=8',
    'data_loader.params.num_workers=8'
])


/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default_training': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


## Launch tensorboard for visualizing training artifacts

In [10]:
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 11158), started 0:00:16 ago. (Use '!kill 11158' to kill it.)

## Launch training (within the notebook)

In [5]:
from nuplan.planning.script.run_training import main as main_train

# Run the training loop, optionally inspect training artifacts through tensorboard (above cell)
main_train(cfg)

INFO:nuplan.planning.script.default_path:Setting default NUPLAN_DATA_ROOT: /home/fla/nuplan/dataset
INFO:nuplan.planning.script.default_path:Setting default NUPLAN_EXP_ROOT: /home/fla/nuplan/exp
Global seed set to 0
INFO:nuplan.planning.script.builders.worker_pool_builder:Building WorkerPool...
INFO:nuplan.planning.utils.multithreading.worker_ray:Starting ray local!
E0224 10:48:23.978921714    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 10:48:24.167851791    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
2022-02-24 10:48:24,788	INFO services.py:1374 -- View the Ray dashboard at http://172.21.2.20:8265
E0224 10:48:24.791417789    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 10:48:24.806185753    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll pol

2022-02-24 10:48:26,140 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:16}  Building experiment folders...
2022-02-24 10:48:26,142 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:18}  Experimental folder: /home/fla/temp/tutorial_nuplan_framework/training_raster_experiment/2022.02.24.10.48.26
2022-02-24 10:48:26,142 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:17}  Building AbstractScenarioBuilder...
2022-02-24 10:48:26,407 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:20}  Building AbstractScenarioBuilder...DONE!
2022-02-24 10:48:26,408 INFO {/home/fla/nuplan-devkit/nuplan/planning/training/experiments/training.py:43}  Building training engine...
2022-02-24 10:48:26,408 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:18}  Building NNModule...
2022-02-24 10:48:26,731 INFO {/home/fla/anaconda3/envs/nupla

Ray objects: 100%|██████████| 12/12 [00:02<00:00,  5.77it/s]


2022-02-24 10:48:32,218 INFO {/home/fla/nuplan-devkit/nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py:195}  Converting scenarios to AbstractScenario...DONE
2022-02-24 10:48:32,218 INFO {/home/fla/nuplan-devkit/nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py:199}  Flattening multi-sample scenarios to single-sample scenarios...
2022-02-24 10:48:32,219 INFO {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:107}  Submitting 12 tasks!


Ray objects: 100%|██████████| 12/12 [00:00<00:00, 1660.29it/s]


...

E0224 10:49:28.403662036    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:28.447379808    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:28.489653896    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:28.531206105    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:28.573445853    7532 fork_posix.cc:70]           For

Validating: 0it [00:00, ?it/s]

E0224 10:49:34.890443976    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:34.934087197    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:34.975954303    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:35.018943243    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:35.061105935    7532 fork_posix.cc:70]           For

Validating: 0it [00:00, ?it/s]

E0224 10:49:41.324659884    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:41.368163407    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:41.410024859    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:41.452399203    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
E0224 10:49:41.494222800    7532 fork_posix.cc:70]           For

## Launch training (command line - alternative)

A training experiment with the above same parameters can be launched alternatively with:
```
$ python nuplan/planning/script/run_training.py \
    experiment_name=raster_experiment \
    py_func=train \
    +training=training_raster_model \
    scenario_builder=nuplan_mini \
    scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=500 \
    scenario_builder.nuplan.scenario_filter.subsample_ratio=0.01 \
    lightning.trainer.params.max_epochs=10 \
    data_loader.params.batch_size=8 \
    data_loader.params.num_workers=8
```

# Simulating a planner <a name="simulation"></a>

## Open-loop simulation
Open-loop simulation aims to evaluate the policy's capabilities to imitate the expert driver's behavior.<br />
This is essentially done through log replay as the policy's predictions do not affect the state of the simulation.

As the policy is not in full control of the vehicle, this type of simulation can only provide a high-level performance overview.

## Closed-loop simulation
Conversely, in closed-loop simulation the policy's actions alter the state of the simulation which tries to closely approximate the real-world system.

The simulation's feedback loop enables a more in-depth evaluation of the policy as compounding errors can cause future observations to significantly diverge from the ground truth.<br />
This is important in measuring distribution shifts introduced due to lack of variance in training examples through pure imitation learning.

Closed-loop simulation is further divided into two categories:
* ego closed-loop simulation with agents replayed from log (open-loop, non reactive)
* ego closed-loop simulation with agents controlled by a rule-based or learned policy (closed-loop, reactive)

## Measuring success
Measuring the success of a planning task and comparing various planning policies is a complicated effort that involves defining metrics across different vertical dimensions and scenario categories.<br />
These metrics include indicators such as vehicle dynamics, traffic rule violations, expert imitation, navigation success etc.<br />
Overall, they aim to capture the policy's ability to control the autonomous vehicle safely yet efficiently without compromising the passenger's comfort.

## Simulation parameters

### Planners

Change the planner model with `planner=X` where `X` is a config yaml defined in the table below. 

| Planner | Description | Config |
| --- | --- | --- |
| Simple Planner | Naive planner that only plans a straight path | `simple_planner` |
| ML Planner | Learning-based planner trained using the nuPlan training framework (see previous section) | `ml_planner` |

## Prepare the simulation config

In [21]:
# Location of path with all simulation configs
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
CONFIG_NAME = 'default_simulation'

# Select the planner and simulation challenge
PLANNER = 'simple_planner'  # [simple_planner, ml_planner]
CHALLENGE = 'challenge_1_open_loop_boxes'  # [challenge_1_open_loop_boxes, challenge_3_closed_loop_nonreactive_agents, challenge_4_closed_loop_reactive_agents]
DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database
    'scenario_builder/nuplan/scenario_filter=all_scenarios',  # initially select all scenarios in the database
    'scenario_builder.nuplan.scenario_filter.scenario_types=[nearby_dense_vehicle_traffic, ego_at_pudo, ego_starts_unprotected_cross_turn, ego_high_curvature]',  # select scenario types
    'scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=10',  # use 10 scenarios per scenario type
    'scenario_builder.nuplan.scenario_filter.subsample_ratio=0.05',  # subsample 20s scenario from 20Hz to 1Hz
]

# Name of the experiment
EXPERIMENT = 'simulation_simple_experiment'

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',
    f'group={SAVE_DIR}',
    f'planner={PLANNER}',
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
])

/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default_simulation': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


## Launch simulation (within the notebook)

In [31]:
from nuplan.planning.script.run_simulation import main as main_simulation

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
# main_simulation(cfg)

# Fetch the filesystem location of the simulation results file for visualization in nuBoard (next section)
parent_dir = Path(SAVE_DIR) / EXPERIMENT

results_dir = (sorted(list(parent_dir.iterdir())))[-1]  # get the child dir
print(list(results_dir.iterdir()))
nuboard_file_1 = [str(file) for file in results_dir.iterdir() if file.is_file() and file.suffix == '.nuboard'][0]
print(nuboard_file_1)

[PosixPath('/home/fla/temp/tutorial_nuplan_framework/simulation_simple_experiment/2022.02.24.11.09.08/log.txt'), PosixPath('/home/fla/temp/tutorial_nuplan_framework/simulation_simple_experiment/2022.02.24.11.09.08/nuboard_1645672154.nuboard'), PosixPath('/home/fla/temp/tutorial_nuplan_framework/simulation_simple_experiment/2022.02.24.11.09.08/simulation'), PosixPath('/home/fla/temp/tutorial_nuplan_framework/simulation_simple_experiment/2022.02.24.11.09.08/metrics')]
/home/fla/temp/tutorial_nuplan_framework/simulation_simple_experiment/2022.02.24.11.09.08/nuboard_1645672154.nuboard


## Launch simulation (command line - alternative)

A simulation experiment can be launched alternatively with:
```
$ python nuplan/planning/script/run_simulation.py \
    +simulation=challenge_1_open_loop_boxes \
    planner=simple_planner \
    scenario_builder=nuplan_mini \
    scenario_builder/nuplan/scenario_filter=all_scenarios \
    scenario_builder.nuplan.scenario_filter.scenario_types="[nearby_dense_vehicle_traffic, ego_at_pudo, ego_starts_unprotected_cross_turn, ego_high_curvature]" \
    scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=10 \
    scenario_builder.nuplan.scenario_filter.subsample_ratio=0.05
```

## Simulate a trained ML planner for comparison

Using the same simulation settings as before, we can simulate a pretrained ML planner and compare the two.

In this example you can take the model you trained earlier.

In [16]:
# Location of path with all simulation configs
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
CONFIG_NAME = 'default_simulation'

# Get the checkpoint of the trained model
last_experiment = sorted(os.listdir(LOG_DIR))[-1]
train_experiment_dir = sorted(Path(LOG_DIR).iterdir())[-1]
checkpoint = sorted((train_experiment_dir / 'checkpoints').iterdir())[-1]

MODEL_PATH = str(checkpoint).replace("=", "\=")

# Name of the experiment
EXPERIMENT = 'simulation_raster_experiment'

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',
    f'group={SAVE_DIR}',
    'planner=ml_planner',
    'model=raster_model',
    'planner.model_config=${model}',  # hydra notation to select model config
    f'planner.checkpoint_path={MODEL_PATH}',  # this path can be replaced by the checkpoint of the model trained in the previous section
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
])

# Run the simulation loop
main_simulation(cfg)

# Fetch the filesystem location of the simulation results file for visualization in nuBoard (next section)
parent_dir = Path(SAVE_DIR) / EXPERIMENT
results_dir = list(parent_dir.iterdir())[-1]  # get the child dir
nuboard_file_2 = [str(file) for file in results_dir.iterdir() if file.is_file() and file.suffix == '.nuboard'][0]

2022-02-24 11:04:20,779 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:126}  Building planners...
2022-02-24 11:04:20,779 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:126}  Building planners...
2022-02-24 11:04:20,780 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:18}  Building NNModule...
2022-02-24 11:04:20,780 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:18}  Building NNModule...
2022-02-24 11:04:21,000 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/timm/models/helpers.py:188}  Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth)
2022-02-24 11:04:21,000 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/timm/models/helpers.py:188}  Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-

Global seed set to 0


2022-02-24 11:04:21,837 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:128}  Building planners...DONE!
2022-02-24 11:04:21,837 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:128}  Building planners...DONE!
2022-02-24 11:04:21,847 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:18}  Building WorkerPool...
2022-02-24 11:04:21,847 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:18}  Building WorkerPool...
2022-02-24 11:04:21,848 WARNING {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:98}  Ray is running, we will shut it down before starting again!
2022-02-24 11:04:21,848 WARNING {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:98}  Ray is running, we will shut it down before starting again!
2022-02-24 11:04:23,537 INFO {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:63}  Starting ray loc

E0224 11:04:23.541703956    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 11:04:23.660419278    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 11:04:23.778319250    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 11:04:24.890796398    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 11:04:25.014354687    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
2022-02-24 11:04:25,726	INFO services.py:1374 -- View the Ray dashboard at http://172.21.2.20:8265
E0224 11:04:25.730838096    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0224 11:04:25.835359242    7532 fork_posix.cc:70]           Fork support is on

2022-02-24 11:04:27,127 INFO {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:95}  Worker: RayDistributed
2022-02-24 11:04:27,127 INFO {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:95}  Worker: RayDistributed
2022-02-24 11:04:27,129 INFO {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:96}  Number of nodes: 1
Number of CPUs per node: 12
Number of GPUs per node: 1
Number of threads across all nodes: 12
2022-02-24 11:04:27,129 INFO {/home/fla/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:96}  Number of nodes: 1
Number of CPUs per node: 12
Number of GPUs per node: 1
Number of threads across all nodes: 12
2022-02-24 11:04:27,132 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:21}  Building WorkerPool...DONE!
2022-02-24 11:04:27,132 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:21}  Building WorkerPool...DONE!


Ray objects: 100%|██████████| 12/12 [00:02<00:00,  5.79it/s]


2022-02-24 11:04:29,291 INFO {/home/fla/nuplan-devkit/nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py:195}  Converting scenarios to AbstractScenario...DONE
2022-02-24 11:04:29,291 INFO {/home/fla/nuplan-devkit/nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py:195}  Converting scenarios to AbstractScenario...DONE
2022-02-24 11:04:29,291 INFO {/home/fla/nuplan-devkit/nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py:226}  Total number of extracted scenarios: 27!
2022-02-24 11:04:29,291 INFO {/home/fla/nuplan-devkit/nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py:226}  Total number of extracted scenarios: 27!
2022-02-24 11:04:29,291 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:46}  Extracting scenarios...DONE!
2022-02-24 11:04:29,291 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:46}  Extracting scenarios...DONE!
2022-02-24 11:04:2

Ray objects: 100%|██████████| 27/27 [02:12<00:00,  4.91s/it]

2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/simulation/runner.py:74}  Number of successful simulations: 27
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/simulation/runner.py:74}  Number of successful simulations: 27
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/simulation/runner.py:75}  Number of failed simulations: 0
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/simulation/runner.py:75}  Number of failed simulations: 0
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:102}  Finished running simulation!
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:102}  Finished running simulation!
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/run_simulation.py:111}  Simulation duration: 00:02:23 [HH:MM:SS]
2022-02-24 11:06:45,796 INFO {/home/fla/nuplan-devkit/nuplan/planning/script

# Visualizing metrics and scenarios <a name="dashboard"></a>

## nuBoard summary

Having trained and simulated planners across various scenarios and driving behaviors, it's time to evaluate them:
* quantitatively, through common and scenario dependent metrics
* qualitatively, through visualization of scenario progression

### nuBoard tabs
To achieve that, nuBoard has 3 core evaluation tabs:
1. Overview - Scalar metrics summary of common and scenario metrics across the following categories:
    * Ego dynamics
    * Traffic violations
    * Expert imitation
    * Planning & navigation
    * Scenario performance
2. Histograms - Histograms over metric statistics for more a granular peek inside each metric focusing on:
    * Metric statistics (e.g. min, max, p90)
3. Scenarios - Low-level scenario visualizations:
    * Time-series progression of a specific metric across a scenario
    * Top-down visualization of the scenario across time for comparing predicted vs. expert trajectories

In addition, there is a main configuration tab for selecting different simulation files for comparing planners/experiments.

<br />

**NOTE**: nuBoard is under heavy developement, overall functionality and aesthetics do not represent the final product!

## Prepare the nuBoard config

In [32]:
# Location of path with all nuBoard configs
CONFIG_PATH = '../nuplan/planning/script/config/nuboard'
CONFIG_NAME = 'default_nuboard'

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
print(nuboard_file_1, nuboard_file_2)
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    'scenario_builder=nuplan_mini',  # set the database (same as simulation) used to fetch data for visualization
    f'simulation_path={[nuboard_file_1, nuboard_file_2]}',  # nuboard file path(s), if left empty the user can open the file inside nuBoard
])

/home/fla/temp/tutorial_nuplan_framework/simulation_simple_experiment/2022.02.24.11.09.08/nuboard_1645672154.nuboard /home/fla/temp/tutorial_nuplan_framework/simulation_raster_experiment/2022.02.24.10.50.57/nuboard_1645671062.nuboard


/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default_nuboard': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


## Launch nuBoard (open in new tab - recommended)

In [33]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

2022-02-24 11:25:53,851 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:17}  Building AbstractScenarioBuilder...
2022-02-24 11:25:53,851 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:17}  Building AbstractScenarioBuilder...
2022-02-24 11:25:53,853 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:20}  Building AbstractScenarioBuilder...DONE!
2022-02-24 11:25:53,853 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:20}  Building AbstractScenarioBuilder...DONE!
2022-02-24 11:25:53,854 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/metric_builder.py:15}  Building metric categories...
2022-02-24 11:25:53,854 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/metric_builder.py:15}  Building metric categories...
2022-02-24 11:25:53,854 INFO {/home/fla/nuplan-devkit/nuplan/planning/script/builders/me

RuntimeError: This event loop is already running

E0224 11:25:54.258639776    7532 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


2022-02-24 11:25:54,895 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET / (127.0.0.1) 92.26ms
2022-02-24 11:25:54,895 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET / (127.0.0.1) 92.26ms
2022-02-24 11:25:54,965 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET /static/js/bokeh.min.js?v=b3e0592a1e90448fa40dcddd5cd5217dcd6b3c7dd368020f6a8e4be4fdd1adbdeb2824eb75ab4f5120c80dc7684f8ff2aa2f8b00d6f393108c96d2f6f2cf0297 (127.0.0.1) 10.62ms
2022-02-24 11:25:54,965 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET /static/js/bokeh.min.js?v=b3e0592a1e90448fa40dcddd5cd5217dcd6b3c7dd368020f6a8e4be4fdd1adbdeb2824eb75ab4f5120c80dc7684f8ff2aa2f8b00d6f393108c96d2f6f2cf0297 (127.0.0.1) 10.62ms
2022-02-24 11:25:54,971 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET /stati

INFO:tornado.access:200 GET / (127.0.0.1) 92.26ms
INFO:tornado.access:200 GET /static/js/bokeh.min.js?v=b3e0592a1e90448fa40dcddd5cd5217dcd6b3c7dd368020f6a8e4be4fdd1adbdeb2824eb75ab4f5120c80dc7684f8ff2aa2f8b00d6f393108c96d2f6f2cf0297 (127.0.0.1) 10.62ms
INFO:tornado.access:200 GET /static/js/bokeh-gl.min.js?v=863c26b3d7cbcf2a0dbf119589404b3ca66734754cd0af1d6e6ca17679ae711126917f171667194a6f04765eba06d9eb2d7d1f2ba7ef8fee420b9244557386f8 (127.0.0.1) 3.59ms
INFO:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=0ede1975746c96e47b24a08c83a45a9282b6524986185f620debadba5132e16c43f941626151d70d779dde7ff63ef84830eb4b277291752aea5176a795b00cec (127.0.0.1) 4.46ms
INFO:tornado.access:200 GET /static/js/bokeh-tables.min.js?v=e075943aa7fac687b27889393d32dde4c6dd9fc8ca7eeb3864ce96011372ea45d3807d8a2d15b322b051170360eda59a5cd9384219c4b150b271b864df4b07d3 (127.0.0.1) 2.88ms
INFO:tornado.access:200 GET /static/js/bokeh-mathjax.min.js?v=9f66f62859aa420d9e3dcf269f16f897179bd90f9d87c8c67998bd9c73476

...

2022-02-24 11:32:38,308 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET / (127.0.0.1) 70.39ms
2022-02-24 11:32:38,308 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET / (127.0.0.1) 70.39ms
2022-02-24 11:32:38,347 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:306}  WebSocket connection closed: code=1001, reason=None
2022-02-24 11:32:38,347 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:306}  WebSocket connection closed: code=1001, reason=None


INFO:tornado.access:200 GET / (127.0.0.1) 70.39ms


2022-02-24 11:32:38,714 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  101 GET /ws (127.0.0.1) 0.43ms
2022-02-24 11:32:38,714 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  101 GET /ws (127.0.0.1) 0.43ms
2022-02-24 11:32:38,715 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:132}  WebSocket connection opened
2022-02-24 11:32:38,715 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:132}  WebSocket connection opened
2022-02-24 11:32:38,715 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:213}  ServerConnection created
2022-02-24 11:32:38,715 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:213}  ServerConnection created


INFO:tornado.access:101 GET /ws (127.0.0.1) 0.43ms


2022-02-24 11:32:51,507 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:306}  WebSocket connection closed: code=1001, reason=None
2022-02-24 11:32:51,507 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:306}  WebSocket connection closed: code=1001, reason=None


## Launch nuBoard (embedded within the notebook - alternative)

In [12]:
from bokeh.io import show, output_notebook
from nuplan.planning.script.run_nuboard import initialize_nuboard

# Make sure that the notebook working directory is "/notebooks" and that Jupyter was launched at the root of the repo
cfg.bokeh.resource_prefix = '/notebooks/nuplan/planning/metrics/board/'  # pass CSS resources to the notebook

# Run the nuBoard
output_notebook()
nuboard = initialize_nuboard(cfg)
show(nuboard.main_page)

ConfigAttributeError: Key 'bokeh' is not in struct
    full_key: bokeh
    object_type=dict

2022-02-22 16:28:08,456 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET / (172.21.6.84) 173.90ms
2022-02-22 16:28:08,456 INFO {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  200 GET / (172.21.6.84) 173.90ms


INFO:tornado.access:200 GET / (172.21.6.84) 173.90ms


2022-02-22 16:28:08,680 ERROR {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:119}  Refusing websocket connection from Origin 'http://172.21.2.20:5006';                       use --allow-websocket-origin=172.21.2.20:5006 or set BOKEH_ALLOW_WS_ORIGIN=172.21.2.20:5006 to permit this; currently we allow origins {'localhost:5006'}
2022-02-22 16:28:08,680 ERROR {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:119}  Refusing websocket connection from Origin 'http://172.21.2.20:5006';                       use --allow-websocket-origin=172.21.2.20:5006 or set BOKEH_ALLOW_WS_ORIGIN=172.21.2.20:5006 to permit this; currently we allow origins {'localhost:5006'}
2022-02-22 16:28:08,682 WARNING {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  403 GET /ws (172.21.6.84) 2.53ms
2022-02-22 16:28:08,682 WARNING {/home/fla/anaconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2239}  403

## Launch nuBoard (command line - alternative)

nuBoard can be launched alternatively with:
```
$ python nuplan/planning/script/run_nuboard.py
```

Simulation files (.nuboard) can be selected under the configuration tab.